In [1]:
import requests
import re 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import cartopy as ctp
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression


In [2]:
def getxlslist():
    RDurl = "https://www.vendeeglobe.org/fr/classement"
    

In [3]:
r = requests.get("https://www.vendeeglobe.org/fr/classement")
html_doc = r.content
soup = BeautifulSoup(html_doc,'html.parser')


In [4]:
soup = BeautifulSoup(requests.get("https://www.vendeeglobe.org/fr/classement").content.decode("utf-8"))
a= soup.findAll('h2', attrs={'class': "rankings__title"})
print(a)
#print(soup)

[<h2 class="rankings__title">Classement</h2>]


In [5]:
soup = BeautifulSoup(requests.get("https://www.vendeeglobe.org/fr/classement").content.decode("utf-8"))
a= soup.findAll('option')
#print(a[0].text)
options = []
for ioption in a:
    #print(ioption.text[1:])
    #print(ioption.text)
    options.append(ioption.text)
#options

In [7]:
FileLink = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_080000.xlsx"
resp = requests.get(FileLink)
with open("vendeeglobe_20210305_080000.xlsx",'wb') as output:
    output.write(resp.content)

In [24]:
import tempfile
from zipfile import ZipFile
import shutil
import os
from fnmatch import fnmatch
import re

def change_in_zip(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)           
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)

change_in_zip("vendeeglobe_20210305_080000.xlsx", 
              name_filter='xl/styles.xml', # the problematic property is found in the style xml files
              change=lambda d: re.sub(b'xxid="\d*"', b"", d))

In [25]:
df = pd.read_excel ("vendeeglobe_20210305_080000.xlsx")

In [26]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Classement du vendredi 05 mars 2021 à 09h00 FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Rang\nRank,Nat. / Voile\nNat. / Sail,Skipper / Bateau\nSkipper / crew,Date d'arrivée\nArrival date,NaN,NaN,NaN,Temps de course\nRace time,NaN,...,NaN,Écarts\nGaps,NaN,NaN,NaN,Sur l'ortho\nOver ortho,NaN,►,Sur le fond\nOver ground,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Au premier\nTo first,NaN,Au précédent\nTo previous,NaN,Vitesse\nSpeed,Distance\nDistance,NaN,Vitesse\nSpeed,Distance\nDistance
4,NaN,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,...,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
5,NaN,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,NaN,...,NaN,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
6,NaN,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,NaN,NaN,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,NaN,...,NaN,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
7,NaN,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,NaN,NaN,NaN,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,NaN,...,NaN,NaN,10h 00min 09s,NaN,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
8,NaN,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,NaN,NaN,NaN,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,NaN,...,NaN,NaN,11h 14min 59s,NaN,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
9,NaN,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,NaN,NaN,NaN,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,NaN,...,NaN,NaN,11h 37min 15s,NaN,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm


In [27]:
df.isnull().all()

Unnamed: 0      True
Unnamed: 1     False
Unnamed: 2     False
Unnamed: 3     False
Unnamed: 4     False
Unnamed: 5      True
Unnamed: 6      True
Unnamed: 7     False
Unnamed: 8     False
Unnamed: 9      True
Unnamed: 10     True
Unnamed: 11     True
Unnamed: 12    False
Unnamed: 13    False
Unnamed: 14    False
Unnamed: 15    False
Unnamed: 16    False
Unnamed: 17    False
Unnamed: 18    False
Unnamed: 19    False
Unnamed: 20    False
dtype: bool

In [28]:
df = df.dropna(axis = 'columns',
         how = 'all')

df = df.dropna(axis = 'index',
         how = 'all')

In [29]:
df[:-4]

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 7,Unnamed: 8,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
1,Classement du vendredi 05 mars 2021 à 09h00 FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rang\nRank,Nat. / Voile\nNat. / Sail,Skipper / Bateau\nSkipper / crew,Date d'arrivée\nArrival date,NaN,Temps de course\nRace time,Écarts\nGaps,NaN,NaN,NaN,Sur l'ortho\nOver ortho,NaN,►,Sur le fond\nOver ground,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Au premier\nTo first,NaN,Au précédent\nTo previous,NaN,Vitesse\nSpeed,Distance\nDistance,NaN,Vitesse\nSpeed,Distance\nDistance
4,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
5,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
6,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
7,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,NaN,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,NaN,10h 00min 09s,NaN,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
8,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,NaN,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,NaN,11h 14min 59s,NaN,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
9,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,NaN,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,NaN,11h 37min 15s,NaN,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm
10,7\nARV,\nFRA 1000,Damien Seguin\nGroupe APICIL,NaN,28/01/2021 12:18:20 FR,80j 21h 58min 20s\n,NaN,18h 13min 34s,NaN,06h 36min 19s,12.5 kts,24365.7 nm,112.9 %,14.2 kts,27512.3 nm
